In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib.pylab import plt
%matplotlib inline

In [2]:
#first merge in price index and reserve data into transaction data then merge into overall data
def get_all_data():
    df = pd.read_csv('../input/properties_2016.csv')
    #read in Los Angeles price index
    lxx_df = pd.read_csv('../input/LXXRSA.csv', parse_dates=[0])
    #read in chinese reserves
    res_df = pd.read_csv('../input/reserves.csv', parse_dates=[0])
    #read in transaction data
    trans_df = pd.read_csv('../input/train_2016.csv', parse_dates=[2])
    #make sure theres no transaction duplicates
    assert trans_df.drop_duplicates().shape[0] == trans_df.shape[0]
    trans_df['merge_date'] = trans_df.transactiondate.apply(lambda v: v + pd.tseries.offsets.MonthBegin(n=0))
    df_all = df.merge(trans_df, on='parcelid')\
    .merge(lxx_df, how='left', left_on='merge_date', right_on='date')\
    .merge(res_df, how='left', left_on='merge_date', right_on='date').drop(['merge_date', 'date_x', 'date_y'], axis=1)
    df_all.to_pickle('../input/all_data.p')
    return df_all

In [3]:
if not os.path.isfile('../input/all_data.p'):
    df = get_all_data()
else:
    df = pd.read_pickle('../input/all_data.p')

In [4]:
df.columns

Index([u'parcelid', u'airconditioningtypeid', u'architecturalstyletypeid',
       u'basementsqft', u'bathroomcnt', u'bedroomcnt', u'buildingclasstypeid',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'decktypeid',
       u'finishedfloor1squarefeet', u'calculatedfinishedsquarefeet',
       u'finishedsquarefeet12', u'finishedsquarefeet13',
       u'finishedsquarefeet15', u'finishedsquarefeet50',
       u'finishedsquarefeet6', u'fips', u'fireplacecnt', u'fullbathcnt',
       u'garagecarcnt', u'garagetotalsqft', u'hashottuborspa',
       u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcounty',
       u'regionidneighborhood', u'regionidzip', u'roomcnt', u'storytypeid',
       u'threequarterbathnbr', u'typeconstructiontyp

dummy creation and some data cleansing

In [5]:
def create_dummies(df, cols):
    return pd.get_dummies(df, columns=cols)

In [28]:
def create_missing(df, cols):
    dfnew = df[cols].applymap(lambda v: 0 if pd.isnull(v) else 1).rename(columns=lambda v: 'miss_' + v)
    return pd.concat([df, dfnew], axis=1)

In [7]:
def fill_zeroes(df, cols):
    df[cols] = df[cols].fillna(0)
    return df

In [8]:
def fill_means(df, cols):
    means = df[cols].apply(np.mean)
    return df.fillna(means)

In [9]:
def get_topn(df, cols, num_remain):
    for c, num in zip(cols, num_remain):
        topn = df.groupby(c).size().sort_values(ascending=False)[:num].index.values
        df[c] = df[c].apply(lambda v: v if v in topn else np.nan)
    return df

In [10]:
def date_to_num(df, cols):
    min_dates = df[cols].apply(np.min)
    for c in cols:
        df['days_' + c] = df[c].apply(lambda v: (v - min_dates[c]).days)
#         df.drop(c, axis=1)
    return df

In [33]:
get_top_n = ['propertyzoningdesc']
category = ['airconditioningtypeid', 'buildingqualitytypeid','heatingorsystemtypeid',
            'propertylandusetypeid', 'hashottuborspa', 'taxdelinquencyflag'] + get_top_n
missing = ['bathroomcnt', 'bedroomcnt', 'roomcnt', 'numberofstories', 'censustractandblock',
                  'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'lotsizesquarefeet', 'garagecarcnt']
zeroes = ['poolcnt', 'fullbathcnt']
use_average = ['bathroomcnt' , 'bedroomcnt', 'finishedsquarefeet12', 'roomcnt', 'yearbuilt',
              'assessmentyear', 'taxamount', 'lotsizesquarefeet', 'garagecarcnt', 'numberofstories']
date_cols = ['transactiondate']

keepcols = set(['parcelid', 'schiller', 'reserves'] + datecols + category + missing + zeroes + use_average)

In [34]:
df_clean = df.loc[:, keepcols]\
.pipe(get_topn, cols=get_top_n, num_remain=[1])\
.pipe(create_dummies, cols=category)\
.pipe(create_missing, cols=missing)\
.pipe(fill_zeroes, cols=zeroes)\
.pipe(fill_means, cols=use_average)\
.pipe(date_to_num, cols=date_cols)

In [35]:
#need to create the submission table
dropcols = ['censustractandblock', 'calculatedfinishedsquarefeet']
df_sorted = df_clean.sort_values('transactiondate').drop(dropcols, axis=1)


In [36]:
df_sorted.to_pickle('cleaned_data.p')